In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.5.1


# Simple attention

간단하게 attention을 수행하는 모델을 만들어 봅시다.

먼저 input token을 생성을 한다.
이때 input은 sequence는 6으로 하고 각 token 마다 길이가 3인 embedding 벡터를 생성한다.


In [3]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [4]:
inputs.shape

torch.Size([6, 3])

이제 여기에 q2에 대해서 attention score를 계산한다.
우리가 목표하는 거는 query token 2에 대해서 input으로 들어오는 prompt들에 대한 context vector를 만드는 방식이다.
여러 방식들이 있을 수 있지만 가장 간단한 방법은 각 embedding vector를 각 query 마다 곱하는 것이다. 이를 통해 각 token 사이의 관계를 담을 수 있다.

In [8]:
q2 = inputs[1]

attn_score_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_score_2[i] = torch.dot(x_i, q2) 

In [9]:
attn_score_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

attn_score_2에는 각 input token 별로 곱해진 tensor를 가지고 있다. 
이들 값들을 normalize할 필요가 있다. 가장 간단한 방법은 전체 합으로 나누는 방식이다.

In [11]:
attn_weights_2_tmp = attn_score_2 / attn_score_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


이를 좀더 training이 가능하고 extreme value들을 다루기 위해 softmax를 사용한다.

In [12]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_score_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [13]:
attn_weights_2 = torch.softmax(attn_score_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


이제 마지막으로 attention weight를 각 input에 곱해지게 되면 context vector가 만들어진다.

In [14]:
context_vec_2 = torch.zeros(q2.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


지금까지는 query 2에 대해서 attention을 수행했는데 사실은 모든 query에 대해서 context vector를 만들어 내야 한다.
query token하나를 기준으로 dot product를 하는 방식은 결국에는 matrix 연산과 같다.
따라서 이를 matrix 연산으로 만들 수 있다.

In [ ]:
attention_score = inputs @ inputs.T

In [16]:
attention_score

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

여기서 주목해야 하는 거는 [6,3] @ [3,6] 으로 하여 [6,6] 모양의 행렬이 생성이 되었다.
즉 이 과정에서 embedding 축이 서로 contraction 되어서 사라지는 걸 알 수 있다.

In [18]:
attention_score = torch.softmax(attention_score, dim = -1)
attention_score

tensor([[0.1739, 0.1723, 0.1719, 0.1596, 0.1593, 0.1630],
        [0.1618, 0.1787, 0.1779, 0.1595, 0.1570, 0.1650],
        [0.1619, 0.1786, 0.1778, 0.1595, 0.1574, 0.1648],
        [0.1628, 0.1735, 0.1730, 0.1632, 0.1600, 0.1675],
        [0.1643, 0.1715, 0.1718, 0.1617, 0.1702, 0.1605],
        [0.1619, 0.1753, 0.1744, 0.1625, 0.1556, 0.1704]])

그리고 query 축이 아닌 k축을 기준으로 softmax를 취한다.
마지막으로 모든 query에 대한 context를 vector를 구하면 된다.

In [19]:
context_vectors = attention_score @ inputs
context_vectors

tensor([[0.4334, 0.5849, 0.5368],
        [0.4335, 0.5948, 0.5350],
        [0.4337, 0.5945, 0.5348],
        [0.4315, 0.5911, 0.5321],
        [0.4375, 0.5847, 0.5280],
        [0.4295, 0.5945, 0.5343]])

# Trainable model

우리는 지금까지 input의 embedding wight만을 이용해서 context vector를 구했다.
하지만 이럴 경우에는 모델은 전적으롬 embedding weight만으로 결정이 된다.
즉 모델만의 사고 방식을 저장할 수 있는 요소가 필요하다. 
이를 위해 모델 weight를 넣었습니다.

우리는 input에 바로 attention을 계산하는 방식으로 했다. 사실은 이를 각 역활마다 weight를 넣을 수 있다.
query weight: token에 대한 weight
key weight: token에 대응하는 각 token들에 대한 weight
value weight: query key 기반해서 값으로 나타내야 값에 대한 weight

In [20]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

In [21]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [24]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [22]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [25]:
attn_score_2 = query_2 @ keys.T 
attn_score_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])

여기서의 d_k는 d_out에 따른 scale을 나눈 파라미터이다.

In [26]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_score_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [28]:
context_vector_2 = attn_weights_2 @ values 
context_vector_2

tensor([0.3061, 0.8210])

# Self attention impl

이제 nn.module 로 attetion module을 만들어 보자

In [32]:
import torch.nn as nn 
import numpy as np

class SelfAttention(nn.Module):
    
    def __init__(self, d_in, d_out, attn_dtype: torch.dtype = torch.float32):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.attn_dtype = attn_dtype
        self.k_w = nn.Parameter(torch.rand(d_in, d_out, dtype=self.attn_dtype))
        self.q_w = nn.Parameter(torch.rand(d_in, d_out, dtype=self.attn_dtype))
        self.v_w = nn.Parameter(torch.rand(d_in, d_out, dtype=self.attn_dtype))
        self.inverse_d_k = np.reciprocal(np.sqrt(d_out))
        
    def forward(self, x: torch.Tensor):
        weighted_q = x @ self.k_w 
        weighted_k = x @ self.q_w
        attention_scores = weighted_q @ weighted_k.T 
        attention_weights = torch.softmax(attention_scores*self.inverse_d_k , dim=-1)
        weighted_v = x @ self.v_w
        context_vectors = attention_weights @ weighted_v
        return context_vectors
         

In [33]:
torch.manual_seed(123)
sa_v1 = SelfAttention(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


nn.Parameter를 바로 만드는 것 보다는 nn.Linear를 만드는 게 좀 더 좋다 왜냐하면
weiht를 train할 때 linear 라는 힌트를 통해 학습을 할 수 있다.

In [36]:
class SelfAttentionV2(nn.Module):
    
    def __init__(self, d_in, d_out, attn_dtype: torch.dtype = torch.float32, qkv_bias=False):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.attn_dtype = attn_dtype
        self.k_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        self.q_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        self.v_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        self.inverse_d_k = np.reciprocal(np.sqrt(d_out))
        
    def forward(self, x: torch.Tensor):
        weighted_q = self.k_w(x) 
        weighted_k = self.q_w(x)
        attention_scores = weighted_q @ weighted_k.T 
        attention_weights = torch.softmax(attention_scores*self.inverse_d_k , dim=-1)
        weighted_v = self.v_w(x)
        context_vectors = attention_weights @ weighted_v
        return context_vectors

In [38]:
torch.manual_seed(123)
sa_v2 = SelfAttentionV2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


# causal mask

지금까지는 입력으로 들어오는 모든 token에 대한 관계를 attention을 통해 계산했다.
하지만 그렇게 하기 어려운 경우가 있다. 바로 입력이 아닌 토큰에 대한 예측이다.
미래에 나오는 토큰에 대해 attention을 구하는 건 말이 되지 않는다. 
따라서 미래 토큰을 제외하고 지금있는 token 들에 대해서 attention을 구하는 게 옳다.
이를 위해서 도입한게 causla mask다.

In [39]:
context_length=inputs.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


각 softmax계산을 취한 query 축에 대해서 다 더한 값이 1을 만족해야한다.

In [75]:
from typing import Optional


class CausalAttention(nn.Module):
    
    def __init__(self, d_in, d_out, attn_dtype: torch.dtype = torch.float32, qkv_bias=False):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.attn_dtype = attn_dtype
        self.k_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        self.q_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        self.v_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        self.inverse_d_k = np.reciprocal(np.sqrt(d_out))
        
    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None):
        b, context_len, _d_in = x.shape
        if mask is None:
            self.register_buffer('mask', torch.triu(torch.ones(context_len, context_len), diagonal=1))
        
        weighted_q = self.k_w(x) 
        weighted_k = self.q_w(x)
        attention_scores = weighted_q @ weighted_k.transpose(1,2)
        mask = self.mask.bool().reshape(1, context_len, context_len).tile((b,1,1))
        print(f"mask: {mask}")
        attention_scores.masked_fill_(mask, -torch.inf)
        attention_weights = torch.softmax(attention_scores*self.inverse_d_k , dim=-1)
        print(f"attention weight: {attention_weights}")
        weighted_v = self.v_w(x)
        context_vectors = attention_weights @ weighted_v
        return context_vectors

In [76]:
torch.manual_seed(123)

batch_size = 3
context_len, embedding = inputs.shape

d_out = 12
ca = CausalAttention(embedding, d_out)

batched_input = inputs.tile((batch_size,)).reshape(-1, context_len, embedding)
context_vecs = ca(batched_input)

print(context_vecs)
print(f"context_vecs.shape: {context_vecs.shape}")

mask: tensor([[[False,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True],
         [False, False, False, False,  True,  True],
         [False, False, False, False, False,  True],
         [False, False, False, False, False, False]],

        [[False,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True],
         [False, False, False, False,  True,  True],
         [False, False, False, False, False,  True],
         [False, False, False, False, False, False]],

        [[False,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True],
         [False, False, False, False,  True,  True],
         [False, False, False, False, False,  True],
         [False, False, False, False, False, False]]])
attention weight: tensor([[[1.0000

# Multi head attention

간단하게 attention을 여러개로 만들어서 해보자는 의미에서 나온 개념

In [77]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, d_in, d_out, num_heads, attn_dtype: torch.dtype = torch.float32, qkv_bias=False):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.attn_dtype = attn_dtype
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.inverse_d_k = np.reciprocal(np.sqrt(d_out))
        
        self.k_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        self.q_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        self.v_w = nn.Linear(d_in, d_out, dtype=self.attn_dtype, bias=qkv_bias)
        
        self.out_proj = nn.Linear(d_out, d_out, dtype=self.attn_dtype)
        
        
    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None):
        b, context_len, _d_in = x.shape
        if mask is None:
            self.register_buffer('mask', torch.triu(torch.ones(context_len, context_len), diagonal=1))
        
        weighted_q = self.k_w(x) 
        weighted_k = self.q_w(x)
        weighted_v = self.v_w(x)
        
        # head view
        # [b, context_len, d_out] -> [b, num_heads, context_len, head_dim]
        keys = weighted_k.view(b, context_len, self.num_heads, self.head_dim).transpose(1,2)
        querys = weighted_q.view(b, context_len, self.num_heads, self.head_dim).transpose(1,2)
        values = weighted_v.view(b, context_len, self.num_heads, self.head_dim).transpose(1,2)
        
        attention_scores = querys @ keys.transpose(2,3)
        mask = self.mask.bool().reshape(1, 1, context_len, context_len).tile((b,1,1,1))
        print(f"mask: {mask}")
        attention_scores.masked_fill_(mask, -torch.inf)
        attention_weights = torch.softmax(attention_scores*self.inverse_d_k , dim=-1)
        print(f"attention weight: {attention_weights}")
        
        # [b, context_len, num_heads, head_dim]
        context_vectors = (attention_weights @ values).transpose(1,2)
        context_vectors = context_vectors.contiguous().view(b, context_len, self.d_out)
        context_vectors = self.out_proj(context_vectors)
        
        return context_vectors

In [78]:
torch.manual_seed(123)

batch_size = 3
context_len, embedding = inputs.shape

d_out = 12
num_heads = 4
ca = MultiHeadAttention(embedding, d_out, num_heads)

batched_input = inputs.tile((batch_size,)).reshape(-1, context_len, embedding)
context_vecs = ca(batched_input)

print(context_vecs)
print(f"context_vecs.shape: {context_vecs.shape}")

mask: tensor([[[[False,  True,  True,  True,  True,  True],
          [False, False,  True,  True,  True,  True],
          [False, False, False,  True,  True,  True],
          [False, False, False, False,  True,  True],
          [False, False, False, False, False,  True],
          [False, False, False, False, False, False]]],


        [[[False,  True,  True,  True,  True,  True],
          [False, False,  True,  True,  True,  True],
          [False, False, False,  True,  True,  True],
          [False, False, False, False,  True,  True],
          [False, False, False, False, False,  True],
          [False, False, False, False, False, False]]],


        [[[False,  True,  True,  True,  True,  True],
          [False, False,  True,  True,  True,  True],
          [False, False, False,  True,  True,  True],
          [False, False, False, False,  True,  True],
          [False, False, False, False, False,  True],
          [False, False, False, False, False, False]]]])
attention w